# Simultaneous generation-classification using LSTM 
Implementation of the paper: Simultaneous Generation-classification using lstm 

Authors: Daniel L. Marino, Kasun Amarasinghe, Milos Manic


In [1]:
#from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
import pickle
import math
from six.moves import range
from six.moves.urllib.request import urlretrieve

from twodlearn.tf_lib.Feedforward import LinearLayer
from twodlearn.tf_lib.Recurrent import *

import sys
working_dir= os.getcwd()

print('Working directory:', working_dir)

Working directory: /home/marinodl/projects/co_generation_classification/sentiment_analysis_imdb


#### Configuration Variables

In [8]:
allow_valid= True
Valid_Percentage= 0.1 # i.e. 10% of the data will be used for validation

pos_net_name= '_pos'
neg_net_name= '_neg'

activation_function='tanh'
num_nodes = [300, 300] #num_nodes: Nodes for the LSTM cell
alpha = 10.0 #0.1 #0.1
beta = 0.1 #0.01 #10000.01
lambda_w = 0.00001

dropout_cons = 0.8

Allow_Bias= False 

learning_rate= 0.005      # 0.001
grad_clip_thresh= 1.1       # 0.00001

current_run= 1
batch_size= 64 #64
num_unrollings= 64 #100

batch_size_val= 64 #len(valid_text_pos)/num_unrollings_val #500
num_unrollings_val= num_unrollings #100

comment='_noDropout_LcLpLcp_64unrol_standarloss'

In [9]:
model_version = 'L'+str(len(num_nodes))
print("num_nodes:",num_nodes)
print("alpha:",alpha, ", beta:",beta,", lambda_w:", lambda_w)
print("num_unrollings:",num_unrollings, ", batch_size:",batch_size,", batch_size_val:", batch_size_val)
print("learning_rate:", learning_rate, 'grad_clip_thresh:', grad_clip_thresh)

num_nodes: [300, 300]
alpha: 10.0 , beta: 0.1 , lambda_w: 1e-05
num_unrollings: 64 , batch_size: 100 , batch_size_val: 100
learning_rate: 0.005 grad_clip_thresh: 1.1


# 1. Load training dataset

In [15]:
vc= pickle.load( open( "imdb_vc.pkl", "rb" ) )
num_inputs=  vc.vocabulary_size
num_outputs= vc.vocabulary_size

dataset= pickle.load( open( "imdb_dataset.pkl", "rb" ) )

dataset.train.set_batch_and_unrollings(batch_size, num_unrollings)
dataset.valid.set_batch_and_unrollings(batch_size_val, num_unrollings_val)

train_x, train_y= dataset.train.next_batch()
print(len(train_x))
print(train_y.shape)
print(train_x[0].shape)

print(vc.keys2text([np.argmax(train_x[i][0,:], 0) for i in range(len(train_x))]))
print(train_y[0])

print(vc.keys2text([np.argmax(train_x[i][50,:], 0) for i in range(len(train_x))]))
print(train_y[50])

valid_x, valid_y= dataset.valid.next_batch()

65
(100, 1)
(100, 3001)
head are discov in the lake steve the but not exactli sharp sheriff is on the case but onli if he s not too busi chase big citi thug travel through the area halfway through the film there suddenli is an scene about a thug rob a store and kill two peopl in the process thi moment appear to take on the complet other side 
[ 0.]
understand whi it s brilliant it stay almost entir faith to s book without be or overli and add to it a that is pure cinemat how mani film adapt of ani author s work can claim that himself even put in a cameo appear toward the end of the film and can you ask for a better than that not onli is it a 
[ 1.]


In [16]:
print('# inputs:',num_inputs)

print('pos train length:',dataset.train.batch_generators[0]._text_size )
print('neg train length:',dataset.train.batch_generators[1]._text_size )

print('pos valid length:',dataset.valid.batch_generators[0]._text_size )
print('neg valid length:',dataset.valid.batch_generators[1]._text_size )

print('pos test length:',dataset.test.batch_generators[0]._text_size )
print('neg test length:',dataset.test.batch_generators[1]._text_size )

# inputs: 3001
pos train length: 2422173
neg train length: 2422173
pos valid length: 269130
neg valid length: 269130
pos test length: 2633361
neg test length: 2633361


# 2. Batch generators

In [17]:
def logprob(predictions, labels):
    """Log-probability of the true labels in a predicted batch."""
    predictions[predictions < 1e-10] = 1e-10 # this is to prevent that log() returns minus infinity
    return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
    """Sample one element from a distribution assumed to be an array of normalized
    probabilities.
    """
    r = random.uniform(0, 1)
    s = 0
    for i in range(len(distribution)):
        s += distribution[i]
        if s >= r:
            return i
    return len(distribution) - 1

def sample(prediction):
    """Turn a (column) prediction into 1-hot encoded samples."""
    p = np.zeros(shape=[1, vc.vocabulary_size], dtype=np.float)
    p[0, sample_distribution(prediction[0])] = 1.0
    return p

def random_distribution():
    """Generate a random column of probabilities."""
    b = np.random.uniform(0.0, 1.0, size=[1, vc.vocabulary_size])
    return b/np.sum(b, 1)[:,None]

Get supervised-batch from positive and negative texts

# 3. Model for positive and negative reviews

In [18]:
class myLstmNet(LstmNet):
    def get_extra_inputs(self, i, h_list, state_list):
        #print('OK:', len(h_list))
        return i
    
    def evaluate_final_output(self, outputs_list, inputs_list, h_list ):
        ''' Calculates the final output of the neural network, usually it is just a linear transformation
        
        outputs_list: list with the outputs from the last lstm cell
        inputs_list: list of inputs to the network
        h_list: list with all hidden outputs from all the cells
        '''
        ''''''
        all_hidden = list()
        #print('n_unrollings:', len(h_list)) # DELETE !!!
        #print('n_layers:', len(h_list[0])) # DELETE !!!
        
        for t in h_list: # go trough each time step
            all_hidden.append( tf.concat(1,t) )
        return self.out_layer.evaluate(tf.concat(0, all_hidden))  
        
        
        # Original:
        #return self.out_layer.evaluate(tf.concat(0, outputs_list))
    
if len(num_nodes)>1:
    n_extra= [num_inputs for i in range(len(num_nodes)+1)]
    n_extra[0]= 0
    n_extra[-1]= sum(num_nodes) - num_nodes[-1]
else:
    n_extra= [0,0]

In [19]:
class ModelSetup:
    
    def __init__( self, pos_net, neg_net, w, b, batch_size, num_unrollings, drop_prob_list, name=''):
    
        # 1. Create placeholders for inputs 
        self.X = list()
        for iaux in range(num_unrollings + 1):
            self.X.append(tf.placeholder(tf.float32, shape=[batch_size, vc.vocabulary_size], 
                                             name= name+'X_i'+str(iaux)+'_All'))
        aux_inputs = self.X[:num_unrollings]
        aux_labels = self.X[1:]  # inputs shifted by one time step.

        # Create a list for store the placeholders for the labels
        self.labels = tf.placeholder(tf.float32, shape=[batch_size, 1])
       

        # -------------------- unrolling of the network --------------------------- # 
        self.pos_unroll, _= pos_net.unrolling_setup( batch_size, num_unrollings, 
                                                     inputs_list= aux_inputs,
                                                     labels_list= aux_labels,
                                                     drop_prob_list= drop_prob_list,
                                                     reset_between_unrollings= True,
                                                   )

        self.neg_unroll, _= neg_net.unrolling_setup( batch_size, num_unrollings, 
                                                     inputs_list= aux_inputs,
                                                     labels_list= aux_labels,
                                                     drop_prob_list= drop_prob_list,
                                                     reset_between_unrollings= True,
                                                   ) 


        # Classifier.
        # error_per_sample is a vector, its shape is changed to have each unrolling in separate columns
        output_pos= tf.reshape(self.pos_unroll.error_per_sample,[num_unrollings,batch_size])  
        output_neg= tf.reshape(self.neg_unroll.error_per_sample,[num_unrollings,batch_size]) 

        output_pos_mean= tf.reduce_mean(output_pos, reduction_indices= 0) 
        output_neg_mean= tf.reduce_mean(output_neg, reduction_indices= 0) 

        output_mean= tf.transpose(tf.pack( [ output_pos_mean, output_neg_mean ]))


        self.logits = tf.nn.xw_plus_b( output_mean , w, b )


        self.error_per_sample= tf.nn.sigmoid_cross_entropy_with_logits( self.logits, self.labels )
        
        # prediction error
        #Lp = tf.reduce_mean( tf.mul(self.labels, output_pos_mean) + tf.mul(self.labels-1, output_neg_mean))
        Lp = tf.reduce_mean( tf.mul(tf.squeeze(self.labels), output_pos_mean) + 
                             tf.mul(tf.squeeze(1-self.labels), output_neg_mean)
                           )
        # counter-prediction penalty
        #
        Lcp = tf.reduce_mean( tf.mul(tf.squeeze(1-self.labels), tf.exp(-output_pos_mean)) + 
                              tf.mul(tf.squeeze(self.labels), tf.exp(-output_neg_mean))
                            )
        
        #Lcp = tf.reduce_mean( tf.mul(tf.squeeze(1-self.labels), tf.reduce_mean(tf.exp(-output_pos), reduction_indices= 0) ) + 
        #                      tf.mul(tf.squeeze(self.labels), tf.reduce_mean(tf.exp(-output_neg), reduction_indices= 0) )
        #                    )
        
        # regularization
        l2_c= tf.nn.l2_loss(w)
        
        #self.loss = tf.reduce_mean( self.error_per_sample ) + alpha*Lp + beta*(1.0/Lcp)
        #self.loss = tf.reduce_mean( self.error_per_sample ) + alpha*Lp - beta*(Lcp)
        self.alpha_r = tf.placeholder(tf.float32)
        self.beta_r = tf.placeholder(tf.float32)
        
        self.loss = tf.reduce_mean( self.error_per_sample ) + self.alpha_r*Lp + self.beta_r*Lcp + lambda_w*l2_c

In [20]:

graph = tf.Graph()
with graph.as_default():
    # For dropout
    drop_prob = tf.placeholder(tf.float32)
    drop_prob_list = [ drop_prob for i in range(len(num_nodes)+1)]
    drop_prob_list[0]= None
     
    # 1. Define positive and negative neural networks
    pos_net= myLstmNet( num_inputs, num_nodes, num_outputs, n_extra= n_extra,
                        afunction=activation_function, 
                        LstmCell= AlexLstmCell,
                        name= pos_net_name)
        
    neg_net= myLstmNet( num_inputs, num_nodes, num_outputs, n_extra= n_extra,
                        afunction=activation_function, 
                        LstmCell= AlexLstmCell,
                        name= neg_net_name)
    
    # Classifier weights and biases.
    #w = tf.Variable(tf.truncated_normal([2, 1], -0.1, 0.1), name=('w_class')) # unconstrained w
    #w = tf.Variable(tf.constant( [[-2.0], [2.0]]), name=('w_class'), trainable=False) # fixed w
    
    #w = tf.Variable(tf.truncated_normal([1, 1], 0.3, 0.4), name=('w_class')) # unconstrained w
    w = tf.Variable(tf.constant( [[0.3]] ), name=('w_class')) # fixed w
    w = tf.concat(0, [-w, w])
    
    b = tf.Variable(tf.zeros([1]), name=('b_class'), trainable=Allow_Bias)
    
    
    
    # 2. Define unrolling for training
    train= ModelSetup( pos_net, neg_net, w, b, batch_size, num_unrollings, 
                       drop_prob_list, 
                       name='train_')
       
    # 3. Define unrolling for validation
    if allow_valid:
        valid= ModelSetup( pos_net, neg_net, w, b, batch_size_val, num_unrollings_val, 
                           drop_prob_list= [None for dummy in range(len(num_nodes))], 
                           name='valid_')

    # 4. Define unrolling for testing
    pos_gen_test, _= pos_net.unrolling_setup( 1, 1, drop_prob_list= [None, None, None, None, None] )
    neg_gen_test, _= neg_net.unrolling_setup( 1, 1, drop_prob_list= [None, None, None, None, None] )
    
          
    
    # 5. Define optimizer    
    # 1. specify the optimizer
    optimizer = tf.train.AdamOptimizer(learning_rate) # ADAM 0.001
    
    # 2. get the gradients and variables
    # grads_and_vars is a list of tuples (gradient, variable). 
    grads_and_vars = optimizer.compute_gradients(train.loss) 
    gradients, v = zip(*grads_and_vars)
    
    # 3. process the gradients
    gradients, _ = tf.clip_by_global_norm(gradients, grad_clip_thresh)  #1.25 #0.025 #0.001(last used)
    # 4. apply the gradients to the optimization procedure
    optimizer = optimizer.apply_gradients( zip(gradients, v) ) # ADAM
    
    # for prediction
    train_pred = tf.nn.sigmoid(train.logits)
    
    # Saver
    saver = tf.train.Saver()

# 4. Train Energy predictor model

In [21]:
reload_pos= False
load_pos_file=  working_dir+"/weights/Weights_LSTM_" + model_version +"_"+ str(num_nodes[0]) + "u_"+ \
                "1r_pos.ckpt"
    
reload_neg= False
load_neg_file= working_dir+"/weights/Weights_LSTM_" + model_version +"_"+ str(num_nodes[0]) + "u_"+ \
                "1r_neg.ckpt"

reload_all= True
load_all_file=  working_dir+"/weights/Weights_LSTM_" + model_version +"_"+ str(num_nodes[0]) + "u_"+ \
                str(current_run-1) +"r_All"+comment+".ckpt";

save_all= True
save_all_file=  working_dir+"/weights/Weights_LSTM_" + model_version +"_"+ str(num_nodes[0]) + "u_"+ \
                str(current_run) +"r_All"+comment+".ckpt";

In [22]:
if current_run==1:
    reload_all= False;
else:
    reload_pos= False;
    reload_neg= False;

In [ ]:
num_steps = 3000 
summary_frequency = 50
n_valid_tests = 50
n_characters_step= num_unrollings*batch_size

aux_print=0
with tf.Session(graph=graph) as session:
    # -------------------------------- Load weigths from file, or initialize variables ------------------------------------
    if reload_all:
        saver.restore(session, load_all_file)
        #session.run( global_step.assign(0) ) # SGD
        
    elif (reload_pos and reload_neg):
        tf.initialize_all_variables().run()
        pos_net.saver.restore(session, load_pos_file)
        neg_net.saver.restore(session, load_neg_file)
        print('Weights for positive and negative networks loaded')
    else:
        tf.initialize_all_variables().run()
        print('Weights Initialized')
    
    # ------------------------------------------- Training loop --------------------------------------------------
    print('step | Train_err, Train_mis | Valid_err, Valid_mis | loss')
    
    mean_loss = 0.0
    mean_mis_pred = 0.0
    for step in range(num_steps):
        # 1. Get next batch
        batch_X, batch_y = dataset.train.next_batch()
        
        # 2. Setup feed dictionary for network 1 and network 2
        feed_dict = dict()
        # For dropout
        feed_dict[drop_prob] = dropout_cons
        # hyperparameters
        #if step<200:
        #    feed_dict[train.alpha_r] = 1.0
        #    feed_dict[train.beta_r] = 0.0
        #else:
        feed_dict[train.alpha_r] = alpha
        feed_dict[train.beta_r] = beta
            
        # Inroduce labels
        feed_dict[train.labels] = batch_y
        # Introduce inputs
        for i in range(num_unrollings+1):
            feed_dict[train.X[i]] = batch_X[i]
        
        
        
        # 3 Run optimizer
        #_, l, lr = session.run( [optimizer, loss_train, learning_rate], feed_dict=feed_dict) # SGD
        _, l, pred_train_aux = session.run( [optimizer, train.loss, train_pred], feed_dict=feed_dict) # ADAM
        
        mean_mis_pred += np.sum( np.not_equal( np.greater(pred_train_aux, 0.5), np.greater(batch_y, 0.5)) )
        mean_loss += l
        
        # --------------------------------------------- logging ------------------------------------------------
        if dataset.train.batch_generators[0]._text_size<aux_print :
            print('+'*80)
            print('')
            aux_print = 0
        else:
            aux_print += n_characters_step
        
        if step % summary_frequency == 0:
            if step > 0:
                mean_loss = mean_loss / summary_frequency
                mean_mis_pred = mean_mis_pred / summary_frequency
            
            '''train mis classification'''        
            prediction_train = session.run( [tf.nn.sigmoid(train.logits)], feed_dict=feed_dict)          
            mis_pred_train = np.sum( np.not_equal( np.greater(prediction_train, 0.5), np.greater(batch_y, 0.5)) )
            
            # ---------- Print loss in validation dataset ---------
            if allow_valid:
                '''valid mis classification'''
                mis_pred_val = 0
                for step_valid in range(n_valid_tests):
                    # 1. Get next batch
                    batch_X_val, batch_y_val = dataset.valid.next_batch()        
                    feed_dict_val = dict()
                    # For dropout
                    feed_dict_val[drop_prob] = 1.0
                    # Inroduce labels
                    feed_dict_val[valid.labels] = batch_y_val
                    # Introduce inputs
                    for i in range(num_unrollings_val+1):
                        feed_dict_val[valid.X[i]] = batch_X_val[i]
                    # 2. Get prediction
                    prediction_val = session.run( [tf.nn.sigmoid(valid.logits)], feed_dict=feed_dict_val)          
                    mis_pred_val = mis_pred_val*step_valid
                    mis_pred_val += np.sum( np.not_equal( np.greater(prediction_val, 0.5), np.greater(batch_y_val, 0.5)) )
                    mis_pred_val = mis_pred_val/(step_valid+1)
                    #Delete from here
                    #print('err=', np.sum( np.not_equal( np.greater(prediction_val, 0.5), np.greater(batch_y_val, 0.5)) ))
                    #to here
                    
                    #if ((mis_pred_val/batch_y_val.shape[0])>0.2) and (step_valid==0):
                    #    break                        
                    #elif (step_valid==0):
                    #    break
                    #    print('Low validation error reached:', (mis_pred_val/batch_y_val.shape[0]))
                
                ''' weights and bias for classficator '''
                # print values for b and w
                b_t= b.eval()
                w_t= w.eval()
                
                print('%d | %f (%f), %d | %f, %d | %f | w:[%f,%f], b:%f' % (step, 
                                                     mis_pred_train/batch_y.shape[0], mean_mis_pred/batch_y.shape[0], mis_pred_train,
                                                     mis_pred_val/batch_y_val.shape[0], mis_pred_val,
                                                     mean_loss,
                                                     w_t[0], w_t[1], b_t[0]
                                                    ))    
                #if(mis_pred_val/batch_y_val.shape[0] < 0.20):
                #    break
            else:
                print('%d | %f, %f | %f' % (step, mis_pred_train/batch_y.shape[0], mis_pred_train, mean_loss))
                      
            mean_loss = 0
            mean_mis_pred = 0
            
    if save_all:
        save_path = saver.save(session, save_all_file)
        print("LSTM cell Model for negative reviews saved in file: %s" % save_path)
        

Weights Initialized
step | Train_err, Train_mis | Valid_err, Valid_mis | loss
0 | 0.000000 (0.450000), 0 | 0.485600, 48 | 81.811356 | w:[-0.295006,0.295006], b:0.000000


## Generate some samples from positive and negative networks

In [ ]:
with tf.Session(graph=graph) as session:
    # Load weigths from file, or initialize variables
    tf.initialize_all_variables().run()
    saver.restore(session, save_all_file)
    print('Weights loaded')
        
    # print values for b and w
    b_t= b.eval()
    w_t= w.eval()
    #b_t= session.run(b)
    print('b=', b_t)
    print('w=', w_t)
    
    
    # For positive network
    print('a' + '-'*30 + 'pos' + '-'*30)
    for _ in range(5):
        feed = sample(random_distribution())
        sentence = vc.prob2char(feed)
        pos_gen_test.reset_saved_out_state.run()
        for _ in range(30):
            prediction = tf.nn.softmax(pos_gen_test.y).eval({pos_gen_test.inputs_list[0]: feed}) # feed is a 1-hot encoding
            feed = sample(prediction)  # sample returns a 1-hot encoding
            sentence += vc.prob2char(feed)
        print(sentence)
    print('a' + '-' * 80)
    
    # For negative network
    print('a' + '-'*30 + 'neg' + '-'*30)
    for _ in range(5):
        feed = sample(random_distribution())
        sentence = vc.prob2char(feed)
        neg_gen_test.reset_saved_out_state.run()
        for _ in range(30):
            prediction = tf.nn.softmax(neg_gen_test.y).eval({neg_gen_test.inputs_list[0]: feed}) # feed is a 1-hot encoding
            feed = sample(prediction)  # sample returns a 1-hot encoding
            sentence += vc.prob2char(feed)
        print(sentence)
    print('a' + '-' * 80)